## 1. Imports

In [2]:
#Imports
import json
import numpy as np
import pandas as pd
import nltk
from nltk import word_tokenize
import re
import lyricsgenius as lg

#Token
genius = lg.Genius('YOUR TOKEN')

## 2. Descargar Albumes

In [3]:
album = genius.search_album("Un Verano Sin Ti", "Bad Bunny")
album.save_lyrics()

YHLQMDLG = genius.search_album("YHLQMDLG", "Bad Bunny")
YHLQMDLG.save_lyrics()

Ultimo_Tour = genius.search_album("EL ÚLTIMO TOUR DEL MUNDO", "Bad Bunny")
Ultimo_Tour.save_lyrics()

LQNIAS = genius.search_album("LAS QUE NO IBAN A SALIR", "Bad Bunny")
LQNIAS.save_lyrics()

X100 = genius.search_album("x 100PRE", "Bad Bunny")
X100.save_lyrics()

## 3. Carga de datos y preprocesamiento

In [4]:
#Cargamos los datos en formato json

Un_Verano_json = json.load(open('Lyrics_UnVeranoSinTi.json'))
YHLQMDLG_json = json.load(open('Lyrics_YHLQMDLG.json'))
Ultimo_Tour_json = json.load(open('Lyrics_ELÚLTIMOTOURDELMUNDO.json'))
LQNIAS_json = json.load(open('Lyrics_LASQUENOIBANASALIR.json'))
X100_json = json.load(open('Lyrics_X100PRE.json'))

In [5]:
#Creamos dataframes vacíos para los datos del json file

Un_Verano_df = pd.DataFrame(columns=['Song', 'Artist','Album', 'Lyrics'])
YHLQMDLG_df = pd.DataFrame(columns=['Song', 'Artist','Album', 'Lyrics'])
Ultimo_Tour_df = pd.DataFrame(columns=['Song', 'Artist','Album', 'Lyrics'])
LQNIAS_df = pd.DataFrame(columns=['Song', 'Artist','Album', 'Lyrics'])
X100_df = pd.DataFrame(columns=['Song', 'Artist','Album', 'Lyrics'])

In [6]:
#Creamos una función que itere sobre los elementos del json y los añada al df.

def jsontodf (data, df):
    time.sleep(1)
    for i in data['tracks']:
        song = i['song']['title']
        artist = i['song']['artist']
        lyrics = i['song']['lyrics']
              
        df = df.append({'Song': song, 'Artist':artist, 'Lyrics':lyrics}, ignore_index=True)
    
    #Llenamos el album con una condición: si x canción es la primera, entonces el album es y.

    if df.Song.iloc[0] == 'Moscow Mule':
        df.fillna('Un Verano Sin Ti', inplace=True)
    elif df.Song.astype(str).iloc[0] == 'Si Veo a Tu Mamá':
        df.fillna('YHLQMDLG', inplace=True)
    elif df.Song.astype(str).iloc[0] =='EL MUNDO ES MÍO':
        df.fillna('EL ULTIMO TOUR DEL MUNDO', inplace=True)
    elif df.Song.astype(str).iloc[0] =='SI ELLA SALE':
        df.fillna('LAS QUE NO IBAN A SALIR', inplace=True)
    elif df.Song.astype(str).iloc[0] =='NI BIEN NI MAL':
        df.fillna('X100pre', inplace=True)
    
    return df
    

In [7]:
#Aplicamos la función a los 5 albumes

Un_Verano_df = jsontodf(Un_Verano_json, Un_Verano_df)
YHLQMDLG_df = jsontodf(YHLQMDLG_json, YHLQMDLG_df)
Ultimo_Tour_df = jsontodf(Ultimo_Tour_json, Ultimo_Tour_df)
LQNIAS_df = jsontodf(LQNIAS_json, LQNIAS_df)
X100_df = jsontodf(X100_json,X100_df)   

In [8]:
#Separamos los elementos en Lyric (la palabra que precede a las letras), y tomamos el segundo elemento, es decir,
#lo que está luego de la intro.

def lyrics_split(df):
    df = df.convert_dtypes(str)
    
    letra = df.Lyrics.apply(lambda x: x.split('Lyrics',1)[1])
    
    df.Lyrics = letra
    
    df = df.convert_dtypes(str)
     
    return df

In [9]:
#Aplicamos la función a los 5 albumes

Un_Verano_df = lyrics_split(Un_Verano_df)
YHLQMDLG_df = lyrics_split(YHLQMDLG_df)
Ultimo_Tour_df = lyrics_split(Ultimo_Tour_df)
LQNIAS_df = lyrics_split(LQNIAS_df)
X100_df = lyrics_split(X100_df)

## 4. Word Counting

In [140]:
#Creamos una función que convierte las letras en lista, separa todas las palabras y hace que sean un elemento separado y
#devuelva otra lista con esas palabras separadas.

#Como las stopwords de ntlk no incluyen algunas palabras de canciones de este género, como yeh, to'... añadimos esas
#palabras que no aportan información a las stopwords de ntlk. Además otras palabras que buscamos limpiar, como coro, estribillo...

#Este código cuenta la frecuencia de cada palabra por separado, sin embargo, puerto rico se repite mucho en sus canciones y 
#lo cuenta por separado. Entonces, como siempre que dice puerto viene seguido de un rico, pero dice rico en otras oportunidades
#(no lo precede un puerto), en la función añado 'rico' luego de puerto en el df, y resto la frecuencia de 'puerto rico' a 'rico'

def word_count(data):
    
    lista = data.Lyrics.tolist()
    
    palabras = ' '
    
    for x in lista:
        palabras += ' ' + x
    
    token=re.findall('\w+', palabras)
       
    words = []
    
    for word in token:
        words.append(word.lower())
    
    sw = nltk.corpus.stopwords.words('spanish')
    sw_bb = ['bad', 'bunny', 'pa', 'ey', 'eh', 'verso', 'coro', 'estribillo', 'interludio'
             , 'puente', 'pre-coro','outro','yeh','refran','letra', 'intro', 'oh','si', 'quiere', 'ah'
            , 'yeah', 'sé', 'vo', 'va','to', 'voy', 'así', 'bien', 'quiero', '25embed', 'also'
            ,'might', 'you', 'dar', 'tó', 'vamo', 'like', '1', 'je','ere', 'hace', '2', '3', '4','72embed', 'quién',
            'uh','wuh','r', 'nah','puedo','pre','hago','viene','d','wouh','ser']
    
    words_ne=[]
    for word in words:
        if word not in sw:
            if word not in sw_bb:
                words_ne.append(word)
    
    nlp_words=nltk.FreqDist(words_ne)
    nlp_words
    
    result = pd.DataFrame(nlp_words.items(), columns=['word', 'frequency'])
       
    result.loc[result.word == 'puerto', 'word'] = 'puerto rico'
      
    pr = result.loc[result.word == 'puerto rico'].frequency
        
    prr = pr.tolist()
        
    prrr = prr[0]
        
    prrrr = result.loc[result.word == 'rico'].frequency - prrr
        
    result.loc[result.word == 'rico', 'frequency'] = prrrr
        
    
    return result

In [141]:
#Esta función hace exactamente lo mismo que la anterior, solo que la usé para albumes que no dice Puerto Rico (x100pre y Ultimo Tour del Mundo)

def word_counter(data):
    
    lista = data.Lyrics.tolist()
    
    palabras = ' '
    
    for x in lista:
        palabras += ' ' + x
    
    token=re.findall('\w+', palabras)
       
    words = []
    
    for word in token:
        words.append(word.lower())
    
    sw = nltk.corpus.stopwords.words('spanish')
    sw_bb = ['bad', 'bunny', 'pa', 'ey', 'eh', 'verso', 'coro', 'estribillo', 'interludio'
             , 'puente', 'pre-coro','outro','yeh','refran','letra', 'intro', 'oh','si', 'quiere', 'ah'
            , 'yeah', 'sé', 'vo', 'va','to', 'voy', 'así', 'bien', 'quiero', '25embed', 'also'
            ,'might', 'you', 'dar', 'tó', 'vamo', 'like', '1', 'je','ere', 'hace', '2', '3', '4','72embed', 'quién'
            ,'uh','wuh','r','nah','puedo','pre','hago','viene','d','wouh','ser']
    
    words_ne=[]
    for word in words:
        if word not in sw:
            if word not in sw_bb:
                words_ne.append(word)
    
    nlp_words=nltk.FreqDist(words_ne)
    nlp_words
    
    result = pd.DataFrame(nlp_words.items(), columns=['word', 'frequency'])
       
    result.loc[result.word == 'puerto', 'word'] = 'puerto rico'
            
    return result

In [142]:
#Aplicamos la función a los df y obtenemos un df con la frecuencia de las palabras.

Un_Verano_words = word_count(Un_Verano_df)
YHLQMDLG_words = word_count(YHLQMDLG_df)
Ultimo_Tour_words = word_counter(Ultimo_Tour_df)
LQNIAS_words = word_count(LQNIAS_df)
X100_words = word_counter(X100_df)